### IMPORTS

In [11]:
import pandas as pd
import numpy as np
import re

### DATA

In [12]:
wl_cams = pd.read_json('../data/WELAB/welab_cameras.json')
wl_lens = pd.read_json('../data/WELAB/welab_lenses.json')
wl_audio = pd.read_json('../data/WELAB/welab_audio.json')
wl_lights = pd.read_json('../data/WELAB/welab_lights.json')

In [13]:
rc_cams = pd.read_json('../data/RC SERVICE/rcservice_cameras.json')
rc_lenses = pd.read_json('../data/RC SERVICE/rcservice_lenses.json')

In [14]:
ov_cams = pd.read_json('../data/OVIDE/ovide_cameras.json')
ov_lights = pd.read_json('../data/OVIDE/ovide_lights.json')
ov_lenses = pd.read_json('../data/OVIDE/ovide_lenses.json')
ov_audio = pd.read_json('../data/OVIDE/ovide_audio.json')

In [15]:
rental_places = pd.read_json('../data/rental_places.json') 

In [16]:
rental_places

,rental_place_id,rental_place_name,address,email,phone,website
0,1,WELAB,"C. de Albasanz, 59, 28037, Madrid",info@welab.es,0034 911823355,https://www.welabplus.com
1,2,RC SERVICE,"Avda. Prado del Espino, 1, 28660, Boadilla del...",info@rcservice.es,0034 911855868,https://www.rcservice.es
2,3,OVIDE,"Av. Montes de Oca, 19, Nave 23, 28703, San Seb...",madrid@ovide.com,0034 915670407,https://www.ovide.com


### TRANSFORMATION

In [23]:
def transform_data(data_frame, rental_places_df):    #this is a function to transform all data in each dataframe at once.
    
    # first I want to convert all column names to lowercase
    data_frame.columns = data_frame.columns.str.lower()

    # and also I want to rename the price a day column.
    data_frame.rename(columns={'price a day': 'price_a_day'}, inplace=True)
    data_frame.rename(columns={'rental': 'rental_place_name'}, inplace=True)

    # I also would like to convert all values in the DataFrame to lowercase
    data_frame = data_frame.map(lambda x: x.lower() if isinstance(x, str) else x)

    # And I need to standardise the rental_place names and IDs
    rental_id_mapping = dict(zip(rental_places_df['rental_place_name'].str.lower(), rental_places_df['rental_place_id']))

    # Update the 'rental_place' column in the DataFrame with rental IDs
    data_frame['rental_place_id'] = data_frame['rental_place_name'].str.lower().map(rental_id_mapping)

    # and lastly drop the original 'rental_place'
    data_frame.drop(columns=['rental_place_name'], inplace=True) 
    
    return data_frame

In [25]:
wl_cams = transform_data(wl_cams, rental_places)

In [26]:
wl_cams

,category,brand,name,price_a_day,link,rental_place_id
0,cameras,phantom,phantom flex 4k 128gb,pedir presupuesto,https://welabplus.com/shop8/camaras/phantom/ph...,1
1,cameras,gopro,gopro hero 11 black edition,85,https://welabplus.com/shop8/camaras/gopro/gopr...,1
2,cameras,gopro,gopro max 360,70,https://welabplus.com/shop8/camaras/gopro/gopr...,1
3,cameras,gopro,gopro hero 10 black edition,75,https://welabplus.com/shop8/camaras/gopro/gopr...,1
4,cameras,gopro,gopro hero 9 black edition,69,https://welabplus.com/shop8/camaras/gopro/gopr...,1
...,...,...,...,...,...,...
56,cameras,blackmagic,blackmagic micro cinema camera,50,https://welabplus.com/shop8/camaras/blackmagic...,1
57,cameras,blackmagic,blackmagic cinema camera 4k,65,https://welabplus.com/shop8/camaras/blackmagic...,1
58,cameras,kodak,kodak super 8 mm,pedir presupuesto,https://welabplus.com/shop8/camaras/kodak/koda...,1
59,cameras,sigma,sigma fp,50,https://welabplus.com/shop8/camaras/camaras-fu...,1


In [27]:
dataframelist = [wl_lens, wl_audio, wl_lights, rc_cams, rc_lenses, ov_audio, ov_cams, ov_lenses, ov_lights]

for dataframe in dataframelist:

    dataframe_aux = transform_data(dataframe, rental_places)
    dataframe_aux.to_json()

In [28]:
wl_lens

,category,brand,type,name,price_a_day,rental_place_name,link
0,LENSES,leitz,lpl-prime,"SET LENTES LEITZ ELSIE (18,25,35,50,75,100 mm)",850,WELAB,https://welabplus.com/shop8/opticas/lentes-ful...
1,LENSES,tokina,pl-prime,"SET x5 LENTES TOKINA VISTA 18,25,35,50,85 mm",225,WELAB,https://welabplus.com/shop8/opticas/lentes-ful...
2,LENSES,tokina,pl-prime,TOKINA VISTA CINEMA PRIME 18mm,50,WELAB,https://welabplus.com/shop8/opticas/lentes-ful...
3,LENSES,tokina,pl-prime,TOKINA VISTA CINEMA PRIME 25mm,50,WELAB,https://welabplus.com/shop8/opticas/lentes-ful...
4,LENSES,tokina,pl-prime,TOKINA VISTA CINEMA PRIME 35mm,50,WELAB,https://welabplus.com/shop8/opticas/lentes-ful...
...,...,...,...,...,...,...,...
457,LENSES,canon,pl-zoom,CANON ZOOM CINE-SERVO 17-120mm,150,WELAB,https://welabplus.com/shop8/opticas/ef-zoom/ca...
458,LENSES,zeiss,pl-zoom,ZEISS ZOOM LWZ.3 21-100mm/T2.9-3.9,110,WELAB,https://welabplus.com/shop8/opticas/e-zoom/zei...
459,LENSES,zeiss,pl-zoom,COMPACT ZOOM CZ2 15-30mm T2.9,200,WELAB,https://welabplus.com/shop8/opticas/lentes-ful...
460,LENSES,zeiss,pl-zoom,COMPACT ZOOM CZ2 28-80mm T2.9,200,WELAB,https://welabplus.com/shop8/opticas/lentes-ful...
